In [1]:
!git add Parser.ipynb
!git commit -m "working"
!git push

[master b88ed09] working
 1 file changed, 526 insertions(+), 878 deletions(-)
 rewrite OpenDataRussia/OrthodoxChurh/Parser.ipynb (80%)
Counting objects: 5, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (5/5), 6.51 KiB | 0 bytes/s, done.
Total 5 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/DmitrySerg/OpenData
   c4dcdf5..b88ed09  master -> master


## Цель
- Собрать базу открытых данных с сайта Московской патриархии о персоналиях, организациях, богослужебных указаниях и храмах, структурировать их и загрузить в хаб открытых данных

## Задача
- проинвентаризировать данные на сайте Московской патриархии
- написать парсеры и выгрузить данные в форматах CSV и JSON
- загрузить данные в хаб открытых данных http://hubofdata.ru в удобном виде.
## Требования
- открытый исходный код в Github под свободной лицензией
- межплатформенный код (возможность запуска на Linux/Windows)
- предусмотреть перенос всех метаданных в CKAN в виде тегов и атрибутов
- переносить все данные в CKAN или на отдельный хостинг (например, в Github)
## Данные
- Персоналии - http://www.patriarchia.ru/db/persons/
- Организации - http://www.patriarchia.ru/db/organizations/
- Богослужебные указания - http://www.patriarchia.ru/bu/
- Храмы - http://map.patriarhia.ru/
## Пожелания
- реализация в виде программы командной строки или веб-приложения
- возможность запуска на MacOSX
- использовать скриптовые языки такие как Python, Perl, R и другие.

In [314]:
import requests      
import numpy as np   
import pandas as pd  
import time          
from bs4 import BeautifulSoup

import os

import string

from tqdm import tqdm_notebook

In [316]:
main_link = "http://www.patriarchia.ru"
link_persons = "http://www.patriarchia.ru/db/persons/"
link_organizations = "http://www.patriarchia.ru/db/organizations/"
link_methodology = "http://www.patriarchia.ru/bu/"
link_churches = "http://map.patriarhia.ru/"

file_path_persons = os.getcwd() + "/data/"

In [321]:
class Persons:
    
    def __init__(self):
        pass
    
    def getSoup(self, link):
        """
            Returns bs4-formatted page
        """
        html = requests.get(link)
        if html.status_code == 200:
            soup = BeautifulSoup(html.content, 'lxml')
        else:
            soup = None
        return soup
    
    def getPersonLinks(self):
        """
            Returns a list with names, links and affiliations    
        """

        persons = getSoup(link_persons)
        persons = persons.find('div', attrs={"class":"main"})
        persons = persons.findAll('div', attrs={'class':'news'})

        dfPersons = []
        for person in persons:
            name = person.find('h4').text
            link = main_link + person.find('a')['href']
            affiliation = person.find('div').text
            dfPersons.append({"name":name, "link":link, "affiliation":affiliation})
        
        return dfPersons

    def getPersonData(self, link):
        """
            Returns a dict with parsed field:value pairs from personal page

            link: string
                link to personal page
        """

        data = {}

        # first - get the page content
        pageContent = getSoup(link)

        try:
            image_link = pageContent.find('div', attrs={'class':'news_img'})
            image_link = image_link.find('img')['src']
            data.update({'image_link':image_link})
        except:
            pass

        # now concentrate on text
        pageContent = pageContent.find('div', attrs={'class':'main'})
        pageContent = pageContent.findAll("dl")

        # second - find all field-value pairs on page
        for text in pageContent:        
            number_of_fields = len(text.findAll('dt'))*2-1
            if number_of_fields>0:
                parsedFields = []
                currentValue = text.find('dt')
                parsedFields.append(currentValue.getText().strip())

                while number_of_fields>0:
                    currentValue = currentValue.nextSibling
                    if currentValue != '\n':
                        value = currentValue.getText().strip()
                        if value:
                            parsedFields.append(value)
                            number_of_fields -= 1

                text = dict(zip(parsedFields[::2], parsedFields[1::2]))
                data.update(text)

        return data
    
    def saveDataFrame(self, final):
        """
            Save final file into csv and json files
        """
        self.df = pd.DataFrame(final)
        self.df.columns = self.df.columns.str.strip(':')
        self.df = self.df[['name','affiliation','Страна','Дата рождения', 
                           'Дата пострига','Дата хиротонии', 'Дата смерти',
                           'День ангела', 'Епархия', 'Место работы', 'Образование', 
                           'Биография', 'Награды', 'Научные труды, публикации', 
                           'E-mail', 'Web-сайт', 'link', 'image_link']]
        self.df.to_csv(file_path_persons+'persons.csv', encoding='utf8')
        seff.df.to_json(file_path_persons+'persons.json')
        
    
    def runParsing(self):
        self.dfPersons = getPersonLinks()
        final = []
        for person in tqdm_notebook(self.dfPersons):
            try:
                data = getPersonData(person['link'])
                person.update(data)
                final.append(person)
                time.sleep(0.7)
            except Exception as e:
                # print(e)                           # DEBUG
                # print('problem',  person['link'])  # DEBUG
                time.sleep(30)
                
        self.saveDataFrame(final)

In [323]:
per = Persons()
per.runParsing()

In [313]:
def getSoup(link):
    """
        Returns bs4-formatted page
    """
    html = requests.get(link)
    if html.status_code == 200:
        soup = BeautifulSoup(html.content, 'lxml')
    else:
        soup = None
    return soup

def getPersonLinks():
    """
        Returns a list with names, links and affiliations    
    """
    
    persons = getSoup(link_persons)
    persons = persons.find('div', attrs={"class":"main"})
    persons = persons.findAll('div', attrs={'class':'news'})
    
    dfPersons = []
    for person in persons:
        name = person.find('h4').text
        link = main_link + person.find('a')['href']
        affiliation = person.find('div').text
        dfPersons.append({"name":name, "link":link, "affiliation":affiliation})
        
    return dfPersons

def getPersonData(link):
    """
        Returns a dict with parsed field:value pairs from personal page
    
        link: string
            link to personal page
    """
    
    data = {}
    
    # first - get the page content
    pageContent = getSoup(link)
    
    try:
        image_link = pageContent.find('div', attrs={'class':'news_img'})
        image_link = image_link.find('img')['src']
        data.update({'image_link':image_link})
    except:
        pass
    
    # now concentrate on text
    pageContent = pageContent.find('div', attrs={'class':'main'})
    pageContent = pageContent.findAll("dl")
    
    # second - find all field-value pairs on page
    for text in pageContent:        
        number_of_fields = len(text.findAll('dt'))*2-1
        if number_of_fields>0:
            parsedFields = []
            currentValue = text.find('dt')
            parsedFields.append(currentValue.getText().strip())

            while number_of_fields>0:
                currentValue = currentValue.nextSibling
                if currentValue != '\n':
                    value = currentValue.getText().strip()
                    if value:
                        parsedFields.append(value)
                        number_of_fields -= 1

            text = dict(zip(parsedFields[::2], parsedFields[1::2]))
            data.update(text)
            
    return data

In [337]:
def getAllLinks(url):
    """
        Returns a list with names, links and affiliations    
        
        url: string
            url to parse data from
    """
    
    links = getSoup(url)
    links = links.find('div', attrs={"class":"main"})
    links = links.findAll('div', attrs={'class':'news'})
    
    dfLinks = []
    for link in links:
        name = link.find('h4').text
        linkParsed = main_link + link.find('a')['href']
        affiliation = link.find('div').text
        dfLinks.append({"name":name, "link":linkParsed, "affiliation":affiliation})
        
    return dfLinks

In [338]:
dfOrganizations = getAllLinks(link_organizations)

In [339]:
dfOrganizations[0]

{'affiliation': 'Епархии',
 'link': 'http://www.patriarchia.ru/db/text/31086.html',
 'name': 'Абаканская епархия'}